In [ ]:
using Catalyst 

rn = @reaction_network begin
    @parameters c₁ c₂
    (c₁), x₁ → 2x₁+x₂
    (c₂), x₁+x₂ → x₂
end

# parameter values
pmap = [:c₁ => 1.0, :c₂ => 1.0]
# initial conditions
u0map = [:x₁ => 20, :x₂ => 10]
# time interval to solve on
tspan = (0., 0.5)

In [ ]:
using MomentClosure, Latexify

eqs2 = generate_raw_moment_eqs(rn, 2)
dm2_eqs = moment_closure(eqs2, "derivative matching")
latexify(dm2_eqs, :closure, print_all=true)

In [ ]:
println(latexify(dm2_eqs, :closure, print_all=true))

In [ ]:
unknowns(dm2_eqs)

In [ ]:
using OrdinaryDiffEqTsit5

oprob = ODEProblem(dm2_eqs, u0map, tspan, pmap)
dm2_sol = solve(oprob, Tsit5(), saveat=0.01);

In [ ]:
μ₀₁ = dm2_sol[2, :]
μ₀₂ = dm2_sol[5, :]
μ₀₃ = (μ₀₁ .^(-3)) .* (μ₀₂ .^3)
dm2_κ₀₃ = 2 .* μ₀₁ .^3 .- 3 .* μ₀₂ .* μ₀₁ .+ μ₀₃;

In [ ]:
eqs3 = generate_raw_moment_eqs(rn, 3)
dm3_eqs = moment_closure(eqs3, "derivative matching")
latexify(dm3_eqs, :closure, print_all=true)

In [ ]:
println(latexify(dm3_eqs, :closure, print_all=true))

In [ ]:
unknowns(dm3_eqs)

In [ ]:
oprob = ODEProblem(dm3_eqs, u0map, tspan, pmap)
dm3_sol = solve(oprob, Tsit5(), saveat=0.01, abstol=1e-8, reltol=1e-8)

μ₀₁ = dm3_sol[2,:]
μ₀₂ = dm3_sol[5,:]
μ₀₃ = dm3_sol[9,:]
dm3_κ₀₃ = 2 .* μ₀₁ .^ 3 - 3 .* μ₀₂ .* μ₀₁ .+ μ₀₃;

In [ ]:
central_eqs3 = generate_central_moment_eqs(rn, 3)
dm3_central_eqs = moment_closure(central_eqs3, "derivative matching")
oprob = ODEProblem(dm3_central_eqs, u0map, tspan, pmap)
dm3_central_sol = solve(oprob, Tsit5(), saveat=0.01, abstol=1e-8, reltol=1e-8);

In [ ]:
dm3_κ₀₃ ≈ dm3_central_sol[9,:]

In [ ]:
using JumpProcesses

dprob = DiscreteProblem(rn, u0map, tspan, pmap)
jprob = JumpProblem(rn, dprob, Direct(), save_positions=(false, false))

ensembleprob  = EnsembleProblem(jprob)
@time sol_SSA = solve(ensembleprob, SSAStepper(), saveat=0.01, trajectories=100000)

In [ ]:
@time ssa_κ₀₃ = get_cumulants(sol_SSA, 3)[0, 3];

In [ ]:
using Plots, LaTeXStrings

plot(dm2_sol.t, dm2_κ₀₃, lw=2, label="2nd order DM")
plot!(dm2_sol.t, dm3_κ₀₃, lw=2, label="3rd order DM")
plot!(dm2_sol.t, ssa_κ₀₃, lw=2, label="SSA")
plot!(ylabel=L"\kappa_{03}", xlabel=L"t", guidefontsize=14, legend=:bottomright)

In [ ]:
#savefig("../docs/src/assets/derivative_matching_cumulant.svg")

In [ ]:
ln2_eqs = moment_closure(eqs2, "log-normal")
latexify(ln2_eqs, :closure, print_all=true)

In [ ]:
println(latexify(ln2_eqs, :closure, print_all=true))

In [ ]:
latexify(dm2_eqs, :closure, print_all=true)

In [ ]:
println(latexify(dm2_eqs, :closure, print_all=true))

In [ ]:
ln3_eqs = moment_closure(eqs3, "log-normal")
latexify(ln3_eqs, :closure, print_all=true)

In [ ]:
println(latexify(ln3_eqs, :closure, print_all=true))

In [ ]:
println(latexify(dm3_eqs, :closure, print_all=true))